## Extraindo características com a VGG16, usando Transfer Learning

<b>Importando as bibliotecas</b>

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import os
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
from skimage.io import imread, imsave
import numpy as np
from keras.models import Model
import pandas as pd
import csv
from glob import glob

<b>Carregando os pesos da VGG16</b>

In [ ]:
model = VGG16(weights='imagenet', include_top=True)

<b>Visualizando as camadas da VGG16</b>

In [ ]:
model.summary()

<b>Utilizando apenas os pesos da última camada como características</b>

In [ ]:
model = Model(input=model.input, output=model.get_layer('fc1').output)

<b>Adquirindo as imagens da base</b>

In [ ]:
path_melanoma = "/home/vitoria/Área de Trabalho/ICV/SINFO/ph2/melanoma/"
path_naomelanoma = "/home/vitoria/Área de Trabalho/ICV/SINFO/ph2/naomelanoma/"

lista_melanoma = glob(path_melanoma+'*.bmp')
lista_naomelanoma = glob(path_naomelanoma+'*.bmp')

<b>Aplicando os pesos da VGG16 nas imagens para obter as características</b>

In [ ]:
features = []

for i in range(len(lista_melanoma)):
    img = image.load_img(lista_melanoma[i], target_size=(224, 224))
    x = image.img_to_array(img)
    
    # Expande o array, inserindo um novo eixo
    x = np.expand_dims(x, axis=0)
    
    # Adequa sua imagem ao formato exigido pelo modelo
    x = preprocess_input(x)
    
    # Aplica a imagem no modelo e retorna as características extraídas
    feature = model.predict(x).reshape(-1)
    
    # Transformando de np pra list
    feature = list(feature)
    
    # Adicionando label na lista
    feature.append(1)
    
    # Adicionando esse vetor de features na matriz de features
    features.append(feature)
    
for i in range(len(lista_naomelanoma)):
    img = image.load_img(lista_naomelanoma[i], target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    feature = model.predict(x).reshape(-1)
    feature = list(feature)
    feature.append(0)
    features.append(feature)

In [ ]:
print(len(features[0]))

<b>Salvando os dados no arquivo libsvm</b>

In [ ]:
def geraSVMfile(rotulo, lista_feat, name_file, path_out, modo):
    ''''''
    arquivo = open(path_out + name_file, modo)
    featureFile = str(rotulo) + " "
    arquivo.write(featureFile)
    for i in range(len(lista_feat)-1):
        linha = str(str(i + 1) + ":" + str(lista_feat[i]) + " ")
        #print('linha ', str(i + 1), " = ", linha)
        arquivo.write(linha)
    arquivo.write('\n')
    arquivo.close()

name = 'VGG16.libsvm'
path_file = '/home/vitoria/Área de Trabalho/ICV/SINFO/resultados/'

for i in range(len(features)):
    classe = features[i][-1]
    geraSVMfile(rotulo=classe, lista_feat=features[i], name_file=name, path_out=path_file, modo='a')

In [ ]:
#print('Salvando...')
#my_df = pd.DataFrame(features)
#my_df.to_csv('/home/vitoria/Área de Trabalho/ICV/SINFO/featuresVGG16.libsvm', index=False, header=False, sep=' ')